# ICU Clinical Handoff Summary Pipeline

EMR 데이터 로드 → 소견 추출 → 해석 (중복 제거/충돌 해소) → 인수인계 요약 생성

이 노트북은 Google Colab에서 실행할 수 있도록 설계되었습니다.

## 1. Setup & Install

In [ ]:
#@title Install Dependencies
!git clone https://github.com/taejun-song/emr-icu-handover.git
%cd emr-icu-handover
!pip install -q huggingface_hub pandas openpyxl pydantic

In [ ]:
#@title Set API Key
import os
from google.colab import userdata
os.environ["HF_TOKEN"] = userdata.get("HF_TOKEN")

In [ ]:
#@title Imports
import json
import pandas as pd
from pathlib import Path
from IPython.display import Markdown, display
from src.config import (
    BASELINE_FILE, INPUT_DATA_FILE, OUTPUT_DIR, EXTRACTIONS_DIR,
    OUTPUT_FRAMEWORK_FILE, SHEET_NAMES, SHEET_NAME_TO_PROMPT,
)
from src.loader import load_emr_file
from src.extractors import extract_all, extract_sheet
from src.interpreter import interpret
from src.synthesizer import synthesize
from src.schemas import ExtractorOutput

## 2. Load EMR Data

아래 셀을 실행하면 파일 업로드 위젯이 나타납니다.
각 슬롯에 해당하는 Excel 파일을 드래그 앤 드롭하거나 클릭하여 업로드하세요.

In [ ]:
import src.config as _cfg
from google.colab import output
from IPython.display import HTML, display
import base64, asyncio

DATA_DIR = Path("data")
DATA_DIR.mkdir(parents=True, exist_ok=True)

FILE_SLOTS = {
    "baseline": {"label": "Input_Baseline", "desc": "환자 기본 정보", "config": "BASELINE_FILE"},
    "data": {"label": "Input_Data", "desc": "EMR 데이터", "config": "INPUT_DATA_FILE"},
    "framework": {"label": "Output_Framework", "desc": "출력 프레임워크", "config": "OUTPUT_FRAMEWORK_FILE"},
}
_uploaded = {}

def _receive_file(slot_id, filename, b64content):
    content = base64.b64decode(b64content)
    dest = DATA_DIR / filename
    dest.write_bytes(content)
    _uploaded[slot_id] = dest
    return f"{filename} ({len(content)/1024:.1f} KB)"

output.register_callback("notebook._receive_file", _receive_file)

display(HTML("""
<style>
.upload-container { display: flex; gap: 12px; flex-wrap: wrap; margin: 8px 0; }
.upload-slot {
  flex: 1; min-width: 180px; border: 2px dashed #ccc; border-radius: 8px;
  padding: 16px; text-align: center; cursor: pointer; transition: all 0.2s;
  background: #fafafa; position: relative;
}
.upload-slot:hover, .upload-slot.dragover { border-color: #4285f4; background: #e8f0fe; }
.upload-slot.done { border-color: #34a853; border-style: solid; background: #e6f4ea; }
.upload-slot .label { font-weight: bold; font-size: 14px; margin-bottom: 4px; }
.upload-slot .desc { font-size: 12px; color: #666; }
.upload-slot .status { font-size: 12px; margin-top: 8px; color: #888; }
.upload-slot.done .status { color: #34a853; font-weight: bold; }
.upload-slot input { display: none; }
</style>
<div class="upload-container" id="upload-container"></div>
<script>
const slots = {
  baseline:  {label: "Input_Baseline", desc: "환자 기본 정보"},
  data:      {label: "Input_Data", desc: "EMR 데이터"},
  framework: {label: "Output_Framework", desc: "출력 프레임워크"}
};
const container = document.getElementById("upload-container");
Object.entries(slots).forEach(([id, s]) => {
  const div = document.createElement("div");
  div.className = "upload-slot";
  div.id = "slot-" + id;
  div.innerHTML = `
    <div class="label">${s.label}</div>
    <div class="desc">${s.desc}</div>
    <div class="status" id="status-${id}">클릭 또는 드래그</div>
    <input type="file" id="input-${id}" accept=".xlsx">`;
  div.addEventListener("click", () => document.getElementById("input-" + id).click());
  div.addEventListener("dragover", e => { e.preventDefault(); div.classList.add("dragover"); });
  div.addEventListener("dragleave", () => div.classList.remove("dragover"));
  div.addEventListener("drop", e => {
    e.preventDefault(); div.classList.remove("dragover");
    if (e.dataTransfer.files.length) handleFile(id, e.dataTransfer.files[0]);
  });
  document.getElementById("input-" + id)?.remove;
  container.appendChild(div);
  div.querySelector("input").addEventListener("change", e => {
    if (e.target.files.length) handleFile(id, e.target.files[0]);
  });
});
async function handleFile(slotId, file) {
  const status = document.getElementById("status-" + slotId);
  const slot = document.getElementById("slot-" + slotId);
  if (!file.name.toLowerCase().endsWith(".xlsx")) {
    status.textContent = "⚠ .xlsx 파일만 가능";
    status.style.color = "#ea4335";
    return;
  }
  status.textContent = "업로드 중...";
  const reader = new FileReader();
  reader.onload = async () => {
    const b64 = btoa(new Uint8Array(reader.result).reduce((s,b) => s + String.fromCharCode(b), ""));
    const result = await google.colab.kernel.invokeFunction("notebook._receive_file", [slotId, file.name, b64], {});
    const msg = result.data["text/plain"].replace(/^'|'$/g, "");
    status.textContent = "✓ " + msg;
    slot.classList.add("done");
  };
  reader.readAsArrayBuffer(file);
}
</script>
"""))
print("⬆ 위 위젯에 3개의 파일을 모두 업로드한 후, 아래 셀을 실행하세요.")

In [ ]:
missing = [info["label"] for sid, info in FILE_SLOTS.items() if sid not in _uploaded]
if missing:
    raise FileNotFoundError(f"업로드되지 않은 파일: {', '.join(missing)}")

for sid, info in FILE_SLOTS.items():
    setattr(_cfg, info["config"], _uploaded[sid])

baseline_sheets = load_emr_file(_cfg.BASELINE_FILE)
data_sheets = load_emr_file(_cfg.INPUT_DATA_FILE)

print(f"Baseline sheets: {list(baseline_sheets.keys())}")
print(f"Data sheets: {list(data_sheets.keys())}")
for name, df in data_sheets.items():
    print(f"  {name}: {len(df)} rows")

## 3. Run Extractors

In [ ]:
extractor_outputs = await extract_all(data_sheets)

print(f"Extracted from {len(extractor_outputs)} sheets:")
for eo in extractor_outputs:
    print(f"  {eo.sheet_name}: {len(eo.findings)} findings")

In [ ]:
EXTRACTIONS_DIR.mkdir(parents=True, exist_ok=True)
for eo in extractor_outputs:
    out_path = EXTRACTIONS_DIR / f"{eo.sheet_name.replace(' ', '_').lower()}.json"
    out_path.write_text(json.dumps(eo.model_dump(), ensure_ascii=False, indent=2), encoding="utf-8")
    print(f"Saved: {out_path.name}")

## 4. Extraction Inspection

In [ ]:
results = {eo.sheet_name: eo for eo in extractor_outputs}

for sheet_name, eo in results.items():
    source_rows = len(data_sheets[sheet_name])
    findings_count = len(eo.findings)
    header = (
        f"### {sheet_name}\n\n"
        f"**Source rows:** {source_rows} | "
        f"**Findings extracted:** {findings_count}\n\n"
    )
    rows = []
    for i, f in enumerate(eo.findings, 1):
        rows.append(f"| {i} | {f.datetime or '—'} | {f.category} | {f.content[:120]}{'…' if len(f.content) > 120 else ''} |")
    table = "| # | Datetime | Category | Content |\n|---|----------|----------|---------|\n" + "\n".join(rows)
    display(Markdown(header + table))
    display(Markdown("---"))

### Coverage Analysis

In [ ]:
coverage_rows = []
for sheet_name in SHEET_NAMES:
    source_rows = len(data_sheets.get(sheet_name, pd.DataFrame()))
    findings = len(results[sheet_name].findings) if sheet_name in results else 0
    pct = (findings / source_rows * 100) if source_rows > 0 else 0.0
    coverage_rows.append({"Sheet": sheet_name, "Source Rows": source_rows, "Findings Extracted": findings, "Coverage %": round(pct, 1)})

coverage_df = pd.DataFrame(coverage_rows)
display(coverage_df)

## 5. Run Interpreter

In [ ]:
interpreter_output = await interpret(extractor_outputs, baseline_sheets)

print(f"Reconciled findings: {len(interpreter_output.reconciled_findings)}")
print(f"Conflicts resolved:  {len(interpreter_output.conflicts_resolved)}")
print(f"Duplicates removed:  {interpreter_output.duplicates_removed}")
print(f"Input findings:      {interpreter_output.metadata.total_input_findings}")
print(f"Output findings:     {interpreter_output.metadata.total_output_findings}")

In [ ]:
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)
interp_path = OUTPUT_DIR / "interpretation.json"
interp_path.write_text(
    json.dumps(interpreter_output.model_dump(), ensure_ascii=False, indent=2),
    encoding="utf-8",
)
print(f"Saved: {interp_path}")

## 6. Interpretation Inspection

In [ ]:
rows = []
for i, rf in enumerate(interpreter_output.reconciled_findings, 1):
    sources = ", ".join(rf.sources)
    note = rf.resolution_note or "—"
    content_preview = rf.content[:100] + ("…" if len(rf.content) > 100 else "")
    rows.append(f"| {i} | {rf.datetime or '—'} | {content_preview} | {sources} | {note} |")

table = (
    "| # | Datetime | Content | Sources | Resolution Note |\n"
    "|---|----------|---------|---------|--------------------|\n"
    + "\n".join(rows)
)
display(Markdown(table))

### Conflict Resolutions

In [ ]:
if not interpreter_output.conflicts_resolved:
    display(Markdown("*No conflicts detected.*"))
else:
    for i, cr in enumerate(interpreter_output.conflicts_resolved, 1):
        md = (
            f"### Conflict {i}\n\n"
            f"**Description:** {cr.description}\n\n"
            f"**Sources:** {', '.join(cr.sources)}\n\n"
            f"**Resolution:** {cr.resolution}"
        )
        display(Markdown(md))

## 7. Run Synthesizer

In [ ]:
synthesizer_output = await synthesize(interpreter_output)

print(f"Findings incorporated: {synthesizer_output.metadata.findings_incorporated}")
print(f"Date range: {synthesizer_output.metadata.date_range}")

## 8. Synthesis Inspection

In [ ]:
framework_sheets = pd.read_excel(OUTPUT_FRAMEWORK_FILE, engine="openpyxl", sheet_name=None)
expected_sections = list(framework_sheets.keys())

summary_text = synthesizer_output.summary.lower()
coverage_rows = []
for section in expected_sections:
    found = section.lower() in summary_text
    coverage_rows.append({"Section": section, "Found in Summary": found})

coverage_df = pd.DataFrame(coverage_rows)
found_count = coverage_df["Found in Summary"].sum()
total = len(coverage_df)
print(f"Section coverage: {found_count}/{total} ({found_count/total*100:.0f}%)\n")
display(coverage_df)

## 9. Final Summary

In [ ]:
display(Markdown(synthesizer_output.summary))

In [ ]:
summary_path = OUTPUT_DIR / "summary.md"
summary_path.write_text(synthesizer_output.summary, encoding="utf-8")
print(f"Saved: {summary_path}")